In [1]:
import numpy as np

time_series = np.load("../../all_data/page15_AttentionMapsTimeSeries/img/Age/PhysicalActivity/Walking/3D/Female/middle/normal/Saliency_Age_PhysicalActivity_Walking_3D_Female_middle_normal_0.npy")

dimension = "PhysicalActivity"
subdimension = "Walking"
sub_subdimension = "3D"
channel = 0

In [3]:
import plotly.graph_objs as go 
from dash_website.datasets import INFORMATION_TIME_SERIES

x_label = INFORMATION_TIME_SERIES[dimension][subdimension][sub_subdimension]["x_label"]
y_label = INFORMATION_TIME_SERIES[dimension][subdimension][sub_subdimension]["y_label"]


scatter = go.Scatter(y=time_series[channel], mode="markers", marker=dict(size=5))

fig = go.Figure(scatter)

fig.update_layout(
    {
        "xaxis": {"title": x_label},
        "yaxis": {"title": y_label},
    }
)

fig